In [2]:
from pathlib import Path
import json
import pandas as pd

# =========================================================
# 0) 경로 설정
#   - NOTEBOOK_DIR: 현재 노트북이 있는 폴더(=현재 작업 폴더)
#   - DATA_DIR: 노트북 폴더 기준 ../data
#   - OUT_DIR: 노트북 폴더 기준 ./csv (없으면 생성)
# =========================================================
NOTEBOOK_DIR = Path.cwd()                 # ✅ 노트북 실행 위치(보통 노트북 파일 있는 폴더)
DATA_DIR = NOTEBOOK_DIR / ".." / "data"   # ✅ 너가 쓰던 ../data 와 동일 의미
ann_dir = (DATA_DIR / "aihub_downloads" / "raw_val_json_edited").resolve()

OUT_DIR = (NOTEBOOK_DIR / "csv").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("NOTEBOOK_DIR:", NOTEBOOK_DIR.resolve())
print("ann_dir:", ann_dir)
print("OUT_DIR:", OUT_DIR)

# =========================================================
# 1) JSON 수집 (하위 폴더 포함)
# =========================================================
ann_paths = list(ann_dir.rglob("*.json"))
print("JSON files:", len(ann_paths))

rows_cat = []   # categories 기준(파일 단위)
rows_ann = []   # annotations 기준(박스 단위)

bad_json = 0

for p in ann_paths:
    # JSON 파싱 (깨진 파일은 스킵)
    try:
        d = json.loads(p.read_text(encoding="utf-8", errors="replace"))
    except json.JSONDecodeError:
        bad_json += 1
        continue

    # 1) categories 정보(파일 단위)
    for c in d.get("categories", []):
        if isinstance(c, dict):
            rows_cat.append({
                "file": str(p),
                "category_id": c.get("id"),
                "category_name": c.get("name"),
                "supercategory": c.get("supercategory"),
            })

    # 2) annotations 정보(박스 단위)
    for a in d.get("annotations", []):
        if isinstance(a, dict):
            rows_ann.append({
                "file": str(p),
                "category_id": a.get("category_id"),
            })

print("bad_json:", bad_json)

df_cat = pd.DataFrame(rows_cat)
df_ann = pd.DataFrame(rows_ann)

# =========================================================
# 2) (A) categories 기준: (id, name)이 몇 개 파일에 등장했는지
# =========================================================
df_cat_summary = (
    df_cat.dropna(subset=["category_id"])
          .groupby(["category_id", "category_name"], dropna=False)
          .agg(files=("file", "nunique"))
          .reset_index()
          .sort_values(["files", "category_id"], ascending=[False, True])
)

# =========================================================
# 3) (B) annotations 기준: category_id가 몇 번(=bbox 몇 개) 등장했는지
#     - name 붙이려고 categories에서 id->name 매핑 생성
# =========================================================
id2name = (
    df_cat.dropna(subset=["category_id"])
          .drop_duplicates(subset=["category_id"])[["category_id", "category_name"]]
)

df_ann_summary = (
    df_ann.dropna(subset=["category_id"])
          .groupby("category_id", dropna=False)
          .size()
          .reset_index(name="bbox_count")
          .merge(id2name, on="category_id", how="left")
          .sort_values(["bbox_count", "category_id"], ascending=[False, True])
)

print("\n[Unique categories in categories[]]:", df_cat_summary.shape[0])
print("[Unique category_id in annotations[]]:", df_ann_summary.shape[0])

# =========================================================
# 4) CSV 저장 (노트북 폴더 기준 ./csv/)
# =========================================================
cat_csv_path = OUT_DIR / "cat_summary_val.csv"

df_cat_summary.to_csv(cat_csv_path, index=False, encoding="utf-8-sig")

print("saved:")
print("-", cat_csv_path)

NOTEBOOK_DIR: /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/docs
ann_dir: /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/data/aihub_downloads/raw_val_json_edited
OUT_DIR: /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/docs/csv
JSON files: 5520
bad_json: 0

[Unique categories in categories[]]: 54
[Unique category_id in annotations[]]: 54
saved:
- /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/docs/csv/cat_summary_val.csv
